In [20]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np

#model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


import regex as re

import matplotlib.pyplot as plt
import seaborn as sns

#preprocessing
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline

In [21]:
df = pd.read_csv('./data/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [22]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## Explorative

In [23]:
df['Ticket'].isna().value_counts()

False    891
Name: Ticket, dtype: int64

In [24]:
df[['Survived']][df['Age'].isna()].value_counts()

Survived
0           125
1            52
dtype: int64

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## Get target and Feature

In [26]:
df = df[['Sex', 'Pclass', 'Fare', 'Survived', 'Age', 'Embarked', 'Parch', 'SibSp', 'Cabin']] #needs to be a dataframe


In [27]:
y = df['Survived']
X = df.drop(['Survived'], axis=1)

## Column Transform

In [28]:
X['Cabin'] = X['Cabin'].isna()

In [29]:
X

,Sex,Pclass,Fare,Age,Embarked,Parch,SibSp,Cabin
0,male,3,7.2500,22.0,S,0,1,True
1,female,1,71.2833,38.0,C,0,1,False
2,female,3,7.9250,26.0,S,0,0,True
3,female,1,53.1000,35.0,S,0,1,False
4,male,3,8.0500,35.0,S,0,0,True
...,...,...,...,...,...,...,...,...
886,male,2,13.0000,27.0,S,0,0,True
887,female,1,30.0000,19.0,S,0,0,False
888,female,3,23.4500,NaN,S,2,1,True
889,male,1,30.0000,26.0,C,0,0,False


In [30]:
#imputing_NumFeature = ['Age']
#imputing_CatFeatures = ['Cabin', 'Embarked']
numerical_features = ['Fare', 'Age', 'SibSp', 'Parch']
categorical_features = ['Sex', 'Pclass', 'Embarked', 'Cabin']


In [31]:
transformer_impute_onehot = make_pipeline(
    (SimpleImputer(strategy="most_frequent")),
    (OneHotEncoder(drop='first', handle_unknown='ignore')),
    )

transformer_impute_onehot

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(drop='first', handle_unknown='ignore'))])

In [32]:
transformer_impute_scale = make_pipeline(
    (SimpleImputer(strategy="most_frequent")),
    (MinMaxScaler())
    )

transformer_impute_scale

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('minmaxscaler', MinMaxScaler())])

In [33]:
# column transformation
transformer = ColumnTransformer([
    ('onehot',transformer_impute_onehot , categorical_features),
    ('scaling', transformer_impute_scale, numerical_features)
],remainder='passthrough')

#transformer

## Train the model

In [35]:
pipeline = make_pipeline(
    (transformer), # step-1 column transformation
     (LogisticRegression())  # step-2 linear fit
    ).fit(X,y)

## Get Training Score

In [37]:
pipeline.score(X,y)

0.8092031425364759

In [38]:
pipeline_night = make_pipeline(
    (transformer), # step-1 column transformation
     (KNeighborsClassifier())  # step-2 linear fit
    ).fit(X,y)

In [39]:
pipeline_night.score(X,y)

0.8473625140291807

In [ ]:
pipeline.score(X,y)

0.7983193277310925

In [ ]:
pipeline_night.score(X,y)

## Test Model

In [ ]:
df = pd.read_csv('./data/test.csv')